In [ ]:
config= type('test', (object,), {})()
config.img_path = "foo"
config.train_csv_file = "train_labels.csv"
config.val_csv_file = "val_labels.csv"
config.conv_base_lr = 5e-4
config.dense_lr = 5e-3
config.lr_decay_rate = 0.95
config.lr_decay_freq = 10
config.train_batch_size = 128
config.val_batch_size = 128
config.decay = True
config.ckpt_path = "foo"
config.epochs = 100
config.early_stopping_patience = 10
config.num_workers = 2

from main import main

main(config)